Table of Contents:

1. [Generalized Sector Overview](#Generalized-Sector-Overview)
    1. [Year to year](#Year-to-year)
    1. [Sector to sector](#Sector-to-sector)
    1. [Inside a sector](#Inside-a-sector)
    1. [Inside an agent](#Inside-an-agent)
1. [Exploring MUSE's data structures and objects](#Exploring-MUSE's-data-structures-and-objects)
    1. [MCA](#MCA)
    1. [Presets sector](#Presets-sector)
    1. [Generalized sector and agents](#Generalized-sector-and-agents)
1. [Less obvious data types and dimensions](#Less-obvious-data-types-and-dimensions)
    1. [Timeslices](#Timeslices)
    1. [Commodity usage](#Commodity-usage)

# Generalized Sector Overview

## Year to year

If we neglect enough details, the overall algorithm to muse can be simplified to the
following **pseudo-code**:

```Python
def main_muse(prices, sectors):    
    for year in range(2010, 2010):
        # fixed point optimization f(x) = x
        while True:
            prices = run_sectors(sectors, prices)
            if convergence(prices):
                break
```

Nominally, only two quantities change from year to year: the prices, and the assets held
(indirectly via the sector's agents) by each sector. Everything else can be recomputed
from these two inputs.

Note: the functionality showcased above is found in `muse.mca.MCA.run`.

## Sector to sector

From the point of view of the MCA, at the most basic level, the sectors as a whole are a
black box which take as input prices and return prices.

At the next level of detail, the sectors take two additional quantities as input, the
consumption and the supply, where consumption is any commodity that has been consumed
during the operation of a prior sector, and the supply is any commodity produced by a
prior sector, as illustrated in the following **pseudo-code**:

$2^n$

```Python
def run_sectors(sectors, prices):
    market = Dataset()
    market.consumption = zeros(...)
    market.supply = zeros(...)
    
    for sector in sectors:
        market, price_update[sector.products] = \
            sector.next(market, prices)
    return price_update
```

In practice, the consumption of one sector becomes the demand of the next. It follows
the first sector in the list is somewhat special. It is a `PresetsSector`. It's only
real input is the current year. It returns a consumption (and optionally, supply and
prices) that depends only on the current year.

Note: The updated prices are not passed from one sector to the next.

Note: This loop is implemented in `muse.mca.single_year_iteration`.

## Inside a sector

A sector is mostly a container of agents. It is responsible for running the agents and
having them interact, and little else. As such, outside of the agents themselves, it
contains only "read-only" data. It's main data is `technologies`, an `xarray` dataset
describing the technologies of the sector. This data should remain strictly read only.

There are four main steps to running a sector, as illustrated in the following
pseudo-code:

```Python
class Sector(AbstractSector):
    def next(market, prices):

        for interaction in self.interactions:
            interaction(self.agents)

        demands = self.prepare_demands(market, prices, self.technologies)

        for agent in self.agents:
            agent.next(market, prices, demands, self.technologies)

        capacity = self.aggregate_capacity(self.agents)
        return self.update_market(capacity, prices, self.technologies)
```

First, agents are allowed to interact, e.g. exchange assets or information. This one of
two steps where data is actually modified. Then the needed capacity, missing demand, and
decommissioned demand are computed. These demands are used in the following step, where
agents are made to invest in future assets. This is the second step where data is
actually modified.  Finally, consumption, supply, and price updates are computed and
returned.

A `PresetsSector` is by necessity much simpler:

```Python
class PresetsSector(AbstractSector):
    def next(...):
        return market.sel(year=current_year)
```

It ignores it's inputs and returns consumption, supply and price updates for the current
year.

Note: The generalized sector above is implemented in `muse.sectors.Sector`,
    whereas the presets sector can be found in
    `muse.preset_sector.PresetsSector`.

## Inside an agent

The main function of an agent is to invest. The investment takes as input the market as
given to the sector, the demands computed by the sector, and the dataset describing the
technologies. It proceeds by computing the share of the demand it should satisfy. Then,
it restricts the search space of technologies it will consider during investment.
Then, it computes a preference (or cost) for each technology. Finally, it matches the
demand it will fulfill with the restricted set of technology, while minimizing the
cost, and under constraints on production for each technology.

```Python
class Agent(AbstractAgent):
    def next(market, demands, prices, technologies):

        demand_share = self.demand_share(
                market, demands, prices, technologies)
        search_space = self.search_space(
                market, prices, technologies, demand_share)
        objectives = self.objectives(
                market, prices, technologies, search_space)
        self.assets += self.demand_matching(
                objectives, market, prices, technologies)
```

The last step above is the only one that modifies data that is retained from one call of
function to the next (e.g. from one simulation year to the next).

In practice, the first step splits the demand into a share for each asset currently held
by the agent. As a result, the search space and objectives are determined independently
for each asset. However, in the last step, when matching demand to new investments,
these constraints are given per technology, rather than per share of the demand. Hence,
the problems are not quite independent.

Note: the implementation for the functionality above can be found in
    :py:class:`muse.agents.agent.Agent` and :py:class:`muse.agents.agent.InvestingAgent`.

## Exploring MUSE's data structures and objects

### MCA

The simplest way to get an idea of how things are organized is to explore the main
data-structures and objects:

In [1]:
from pathlib import Path
from muse import logger
from muse import examples

logger.setLevel(0)
mca = examples.model()

Above, we've quieted the log since we are running things interactively.
The main input file follows the [TOML](https://github.com/toml-lang/toml) format. It
contains all the information necessary for a creating a basic simulation with two
sectors: a `PresetsSector` which initializes the demand for a generalized sector
modelling  the commercial sector.

In [2]:
mca.sectors

The sectors are listed in the order they run. The priority is given via a `priority`
attribute in the TOML file. For convenience, the MCA object contains an initial market:

In [3]:
mca.market.sel(commodity="electricity")

<xarray.Dataset>
Dimensions:       (region: 1, timeslice: 6, year: 8)
Coordinates:
    commodity     <U11 'electricity'
  * region        (region) object 'USA'
    units_prices  object 'MUS$2010/GWh'
  * timeslice     (timeslice) object ('all-year', 'all-week', 'night') ... ('all-year', 'all-week', 'evening')
  * year          (year) int64 2020 2025 2030 2035 2040 2045 2050 2055
Data variables:
    prices        (timeslice, region, year) float64 0.0702 0.07897 ... 0.09123
    exports       (timeslice, region, year) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    imports       (timeslice, region, year) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    static_trade  (timeslice, region, year) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    supply        (timeslice, region, year) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    consumption   (timeslice, region, year) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

### Presets sector

In this instance, the preset sector only contains only consumption information for the
residential sector. It supplies nothing and does not modify prices:

In [4]:
from muse.commodities import is_enduse
presets = mca.sectors[0].presets

assert (presets.supply == 0).all()
assert not is_enduse(presets.comm_usage).any()
presets

<xarray.Dataset>
Dimensions:          (commodity: 5, region: 1, timeslice: 6, year: 2)
Coordinates:
    represent_hours  (timeslice) int64 1460 1460 1460 1460 1460 1460
  * timeslice        (timeslice) MultiIndex
  - month            (timeslice) object 'all-year' 'all-year' ... 'all-year'
  - day              (timeslice) object 'all-week' 'all-week' ... 'all-week'
  - hour             (timeslice) object 'night' 'morning' ... 'evening'
  * commodity        (commodity) <U11 'electricity' 'gas' 'heat' 'CO2f' 'wind'
  * year             (year) int64 2020 2050
  * region           (region) object 'USA'
    comm_usage       (commodity) int64 0 0 0 0 0
Data variables:
    consumption      (year, commodity, region, timeslice) float64 0.0 ... 0.0
    costs            (year, commodity, region, timeslice) float64 0.0 ... 0.0
    supply           (year, commodity, region, timeslice) float64 0.0 ... 0.0

In the above we show the supply is zero for all years, commodities, regions, and
timeslices. We also show that the preset sector does not contain "enduse" commodities,
where an enduse is a commodity produced by the sector.

### Generalized sector and agents

The generalized sector contains two forms of data. On the one hand, we have the data
describing the technologies. This is data should only be read and never modified:

In [5]:
residential = mca.sectors[1]
assert residential.name == "residential"
residential.technologies

<xarray.Dataset>
Dimensions:                (commodity: 4, region: 1, technology: 2, year: 2)
Coordinates:
  * technology             (technology) object 'gasboiler' 'heatpump'
  * region                 (region) object 'USA'
  * year                   (year) int64 2020 2060
  * commodity              (commodity) object 'electricity' 'gas' 'heat' 'CO2f'
    comm_usage             (commodity) object CommodityUsage.ENERGY|CONSUMABLE ... CommodityUsage.ENVIRONMENTAL|PRODUCT
Data variables:
    level                  (technology, region) object 'fixed' 'fixed'
    cap_par                (technology, region) float64 3.8 8.867
    cap_exp                (technology, region) int64 1 1
    fix_par                (technology, region) int64 0 0
    fix_exp                (technology, region) int64 1 1
    var_par                (technology, region) int64 0 0
    var_exp                (technology, region) int64 1 1
    max_capacity_addition  (technology, region) int64 100 100
    max_capacity_growth    (technology, region) int64 100 100
    total_capacity_limit   (technology, region) float64 3.937e+03 3.937e+03
    technical_life         (technology, region) int64 10 10
    utilization_factor     (technology, region) int64 1 1
    scaling_size           (technology, region) float64 1.89e-06 1.89e-06
    efficiency             (technology, region) int64 86 86
    availability           (technology, region) int64 1 1
    interest_rate          (technology, region) float64 0.1 0.1
    type                   (technology, region) object 'energy' 'energy'
    fuel                   (technology) object 'gas' 'electricity'
    enduse                 (technology) object 'heat' 'heat'
    agent_share_2          (technology, region) int64 1 1
    tech_type              (technology) object 'energy' 'energy'
    fixed_outputs          (commodity, technology, region, year) float64 0.0 ... 0.0
    flexible_outputs       (commodity, technology, region, year) float64 0.0 ... 0.0
    commodity_units        (commodity) object 'GWh/PJ' 'PJ/PJ' 'PJ/PJ' 'kt/PJ'
    fixed_inputs           (commodity, technology, region, year) float64 0.0 ... 0.0
    flexible_inputs        (commodity, technology, region, year) float64 0.0 ... 0.0
    comm_name              (commodity) object 'Electricity' ... 'CO2fuelcomsbustion'
    emmission_factor       (commodity) float64 0.0 56.1 0.0 0.0
    heat_rate              (commodity) int64 1 1 1 1
    unit                   (commodity) object 'GWh' 'PJ' 'PJ' 'kt'

On the other hand, the generalized sector also holds the agents themselves:

In [6]:
residential.agents

[<muse.agent.Agent at 0x11c1cb310>, <muse.agent.Agent at 0x11c1d9790>]

In turn, each agent holds assets:

In [7]:
residential.agents[0].assets

<xarray.Dataset>
Dimensions:     (asset: 1, year: 2)
Coordinates:
    region      <U3 'USA'
  * year        (year) int64 2020 2050
    technology  (asset) object 'gasboiler'
    installed   (asset) int64 2020
Dimensions without coordinates: asset
Data variables:
    capacity    (year, asset) int64 0 0

Currently, the `assets` dataset only contains one attribute, the `capacity`. At a later
point, it could contain further information, such as the amount produced by an asset,
e.g. to create finite resource constraints. In any case, the sector should not modify
these assets directly, but rather leave it to the agents to
handle. For convenience, the sector has access to the capacity of all assets aggregated
across agents:

In [8]:
residential.capacity

<xarray.DataArray 'capacity' (year: 7, asset: 1)>
array([[10.],
       [ 5.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.]])
Coordinates:
    region      <U3 'USA'
  * year        (year) int64 2020 2025 2030 2035 2040 2045 2050
    technology  (asset) <U9 'gasboiler'
    installed   (asset) int64 2020
Dimensions without coordinates: asset

This aggregation is computed on the fly, whenever the attribute `capacity` is accessed
(it should be used somewhat sparingly). Hence, it cannot be modified:

In [9]:
from pytest import raises
with raises(Exception):
    residential.capacity = 0

The code above would fail if an assertion was not raised.
But **beware**, because python is python and this read-only magic does not extend too
far. The first line below will **not** modify the capacity, even though, superficially,
it looks like it does.

In [10]:
residential.capacity[:] = 0
assert (residential.capacity != 0).any()

In summary, agents can modify the assets they hold, sectors do not hold assets directly,
and hence cannot modify anything. Instead, they should compute whatever quantity they
need at the point of need, without storing the result:

In [11]:
from muse.quantities import decommissioning_demand
decommissioning_demand(residential.technologies, residential.capacity)

<xarray.DataArray (asset: 1, year: 6, commodity: 4)>
array([[[ 0.,  0.,  5.,  0.],
        [ 0.,  0., 10.,  0.],
        [ 0.,  0., 10.,  0.],
        [ 0.,  0., 10.,  0.],
        [ 0.,  0., 10.,  0.],
        [ 0.,  0., 10.,  0.]]])
Coordinates:
    region      <U3 'USA'
    technology  (asset) <U9 'gasboiler'
    installed   (asset) int64 2020
  * year        (year) int64 2025 2030 2035 2040 2045 2050
  * commodity   (commodity) object 'electricity' 'gas' 'heat' 'CO2f'
    comm_usage  (commodity) object CommodityUsage.ENERGY|CONSUMABLE ... CommodityUsage.ENVIRONMENTAL|PRODUCT
Dimensions without coordinates: asset

In [12]:
from muse.quantities import annual_levelized_cost_of_energy
annual_levelized_cost_of_energy(mca.market.prices.sel(year=2020), residential.technologies)

<xarray.DataArray (timeslice: 6, technology: 2, region: 1, year: 2)>
array([[[[14.39354189, 14.39354189]],

        [[ 9.42617956,  9.42617956]]],


       [[[14.39354189, 14.39354189]],

        [[ 9.42617956,  9.42617956]]],


       [[[14.39354189, 14.39354189]],

        [[ 9.42617956,  9.42617956]]],


       [[[14.39354189, 14.39354189]],

        [[ 9.42617956,  9.42617956]]],


       [[[14.39354189, 14.39354189]],

        [[ 9.42617956,  9.42617956]]],


       [[[14.39354189, 14.39354189]],

        [[ 9.42617956,  9.42617956]]]])
Coordinates:
  * timeslice   (timeslice) object ('all-year', 'all-week', 'night') ... ('all-year', 'all-week', 'evening')
  * region      (region) object 'USA'
  * technology  (technology) object 'gasboiler' 'heatpump'
  * year        (year) int64 2020 2060

# Less obvious data types and dimensions

Most of the data structures are fairly self-explanatory. However, there are at least two
types and attendant operations that are less well known. These mean to represent
timeslices and commodity usage.

## Timeslices

Unlike most other dimensions, the individual components of a timeslice are not
equivalent one to another. The number of hours in summer weekday evenings is not
equivalent to the number of hours of winter weekend mornings. Hence, we cannot always
rely on `numpy`'s (and `xarray`) [broadcasting
behaviors](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html).
More specifically, if we are not careful, the code will broadcast automatically
and silently, yielding garbarge results.

There are two main kinds of quantities with respect to timeslices:

- Intensive quantitities, like prices, can broadcasted without care: prices for
a whole period is the average over each sub-period.
- Extensive quantities, like costs, cannot be broadcasted: the costs over a
whole period is the sum of the costs over the sub-periods, weighted by the
number of hours.

Lets first examine the timeslice dimension available in the MCA market:

In [13]:
timeslice = mca.market.timeslice
timeslice

<xarray.DataArray 'timeslice' (timeslice: 6)>
array([('all-year', 'all-week', 'night'), ('all-year', 'all-week', 'morning'),
       ('all-year', 'all-week', 'afternoon'),
       ('all-year', 'all-week', 'early-peak'),
       ('all-year', 'all-week', 'late-peak'),
       ('all-year', 'all-week', 'evening')], dtype=object)
Coordinates:
  * timeslice  (timeslice) object ('all-year', 'all-week', 'night') ... ('all-year', 'all-week', 'evening')

This contains three levels: the season, the days of the week, and the hours of the day.
Lets create a few arrays to press point home. We will have the prices and runnings costs
for a set of technologies, as well as the power of each technology per time timeslice. 

In [14]:
from xarray import DataArray
techs = ['batmobile', 'batbike', 'batscooter', 'batcomputer', 'batnotebook', 'batbat']
prices = DataArray([3, 2, 1, 4, 6, 4], coords={'technology': techs}, dims='technology')
costs = prices.copy(data=[3, 2, 1, 2, 2, 2])
power = (
    DataArray([1, 1, 2, 2, 3, 3], coords={'timeslice': timeslice}, dims='timeslice')
    * prices.copy(data=[5, 7, 10, 11, 12, 13])
)
power

<xarray.DataArray (timeslice: 6, technology: 6)>
array([[ 5,  7, 10, 11, 12, 13],
       [ 5,  7, 10, 11, 12, 13],
       [10, 14, 20, 22, 24, 26],
       [10, 14, 20, 22, 24, 26],
       [15, 21, 30, 33, 36, 39],
       [15, 21, 30, 33, 36, 39]])
Coordinates:
  * timeslice   (timeslice) object ('all-year', 'all-week', 'night') ... ('all-year', 'all-week', 'evening')
  * technology  (technology) <U11 'batmobile' 'batbike' ... 'batbat'

The objective is to figure out the cost and prices per unit of power, and, importantly,
per timeslice. However `prices` nor `costs` above are expressed for the whole
year. How can we extend them consistently to include timeslices?

Since `prices` is an intensive quantity, prices for each timeslice can be meaningfully
said to be the same as for the whole year:

In [15]:
prices_per_ts = prices.expand_dims(timeslice=timeslice)
prices_per_gw = prices_per_ts / power
prices_per_gw

<xarray.DataArray (timeslice: 6, technology: 6)>
array([[0.6       , 0.28571429, 0.1       , 0.36363636, 0.5       ,
        0.30769231],
       [0.6       , 0.28571429, 0.1       , 0.36363636, 0.5       ,
        0.30769231],
       [0.3       , 0.14285714, 0.05      , 0.18181818, 0.25      ,
        0.15384615],
       [0.3       , 0.14285714, 0.05      , 0.18181818, 0.25      ,
        0.15384615],
       [0.2       , 0.0952381 , 0.03333333, 0.12121212, 0.16666667,
        0.1025641 ],
       [0.2       , 0.0952381 , 0.03333333, 0.12121212, 0.16666667,
        0.1025641 ]])
Coordinates:
  * timeslice   (timeslice) object ('all-year', 'all-week', 'night') ... ('all-year', 'all-week', 'evening')
  * technology  (technology) <U11 'batmobile' 'batbike' ... 'batbat'

Above, we went the long way and first expanded `prices` to include timeslices, with the
prices reproduced for each timeslice. Thanks to `numpy`'s broadcasting, we could simply
do:

In [16]:
from xarray.testing import assert_allclose
assert_allclose(prices / power, prices_per_gw.T)

However, the same is not true of costs. We expect that the cost for longer time period
should be larger than for a shorter time period. That is to say, costs are an extensive
quantity. Hence, broadcasting won't do. We first need to expand the costs to an array
with timeslices, such that the sum of all running costs remain the same:

In [17]:
from muse.timeslices import convert_timeslice
costs_per_ts = convert_timeslice(costs, timeslice)
assert_allclose(costs, costs_per_ts.sum("timeslice"))
costs_per_ts

<xarray.DataArray (timeslice: 6, technology: 6)>
array([[0.5       , 0.33333333, 0.16666667, 0.33333333, 0.33333333,
        0.33333333],
       [0.5       , 0.33333333, 0.16666667, 0.33333333, 0.33333333,
        0.33333333],
       [0.5       , 0.33333333, 0.16666667, 0.33333333, 0.33333333,
        0.33333333],
       [0.5       , 0.33333333, 0.16666667, 0.33333333, 0.33333333,
        0.33333333],
       [0.5       , 0.33333333, 0.16666667, 0.33333333, 0.33333333,
        0.33333333],
       [0.5       , 0.33333333, 0.16666667, 0.33333333, 0.33333333,
        0.33333333]])
Coordinates:
  * timeslice   (timeslice) object ('all-year', 'all-week', 'night') ... ('all-year', 'all-week', 'evening')
  * technology  (technology) <U11 'batmobile' 'batbike' ... 'batbat'

Then, and only then can we divide by the power to get the quantity we are looking for:

In [18]:
costs_per_gw = costs_per_ts / power

In order to make usage symmetric, we can explicitly converts any quantity to use
timeslices, as long as we know whether it is intensive or extensive:

In [19]:
from muse.timeslices import QuantityType
convert_timeslice(prices, timeslice, QuantityType.INTENSIVE)

<xarray.DataArray (technology: 6)>
array([3, 2, 1, 4, 6, 4])
Coordinates:
  * technology  (technology) <U11 'batmobile' 'batbike' ... 'batbat'

In this case, the function does not even bother adding timeslices, since it knows
`numpy`'s default broadcasting behavior is sufficient. More to the point, this is useful
when converting between different timeslices. Above, the MCA uses a fairly coarse set of
timeslices. However, one of it's sectors relies on a finer set of timeslices:

In [20]:
mca.sectors[1].timeslices

MultiIndex([('all-year', 'all-week',      'night'),
            ('all-year', 'all-week',    'morning'),
            ('all-year', 'all-week',  'afternoon'),
            ('all-year', 'all-week', 'early-peak'),
            ('all-year', 'all-week',  'late-peak'),
            ('all-year', 'all-week',    'evening')],
           names=['month', 'day', 'hour'])

We can convert either `prices_per_ts` or `costs_per_ts` from coarser to finer sets using
the same function:

In [21]:
from xarray import Dataset
finer = Dataset(
    {
        'costs': convert_timeslice(
            costs_per_ts,
            mca.sectors[1].timeslices,
            QuantityType.EXTENSIVE
        ),
        'prices': convert_timeslice(
            prices_per_ts,
            mca.sectors[1].timeslices,
            QuantityType.INTENSIVE
        ),
    }
)
finer

<xarray.Dataset>
Dimensions:     (technology: 6, timeslice: 6)
Coordinates:
  * timeslice   (timeslice) MultiIndex
  - month       (timeslice) object 'all-year' 'all-year' ... 'all-year'
  - day         (timeslice) object 'all-week' 'all-week' ... 'all-week'
  - hour        (timeslice) object 'night' 'morning' ... 'late-peak' 'evening'
  * technology  (technology) <U11 'batmobile' 'batbike' ... 'batbat'
Data variables:
    costs       (timeslice, technology) float64 0.5 0.3333 ... 0.3333 0.3333
    prices      (timeslice, technology) float64 3.0 2.0 1.0 4.0 ... 4.0 6.0 4.0

## Commodity usage

Depending on the sector a commodity can be a product, or a consumable. Or indeed it
could be both at the same time in the same sector. And it could be more than that. A
commodity can be characterised by whether or not it is a fuel, or whether or not it is
environmental. In short, a number of different flags can be attached to a commodity. To
represent this superposition of flags, python recommends using the
[Flag](https://docs.python.org/3/library/enum.html#flag) class, or it's derived type,
the [IntFlag](https://docs.python.org/3/library/enum.html#intflag) class. We make use of
the latter:

In [22]:
from muse.commodities import CommodityUsage
flags = [flag for flag in CommodityUsage]
flags

[<CommodityUsage.OTHER: 0>,
 <CommodityUsage.CONSUMABLE: 1>,
 <CommodityUsage.PRODUCT: 2>,
 <CommodityUsage.ENVIRONMENTAL: 4>,
 <CommodityUsage.ENERGY: 8>]

The code above lists all current base flags. As can be seen, they each have values of
the form $2^n$. In a binary world, that means these flags can be combined:

In [23]:
CommodityUsage.PRODUCT | CommodityUsage.CONSUMABLE | CommodityUsage.ENVIRONMENTAL

<CommodityUsage.ENVIRONMENTAL|PRODUCT|CONSUMABLE: 7>

This is exactly how each sector defines it's commodities:

In [24]:
residential.technologies.comm_usage

<xarray.DataArray 'comm_usage' (commodity: 4)>
array([<CommodityUsage.ENERGY|CONSUMABLE: 9>,
       <CommodityUsage.ENERGY|CONSUMABLE: 9>,
       <CommodityUsage.ENERGY|PRODUCT: 10>,
       <CommodityUsage.ENVIRONMENTAL|PRODUCT: 6>], dtype=object)
Coordinates:
  * commodity   (commodity) object 'electricity' 'gas' 'heat' 'CO2f'
    comm_usage  (commodity) object CommodityUsage.ENERGY|CONSUMABLE ... CommodityUsage.ENVIRONMENTAL|PRODUCT

In order to help filter commodities, a number of functions are provided in the
`muse.commodities` module. For instance, we can select end-use commodities, i.e.
commodities that a product but are not environmental:

In [25]:
from muse.commodities import is_enduse
residential.technologies.sel(commodity=is_enduse(residential.technologies.comm_usage))

<xarray.Dataset>
Dimensions:                (commodity: 1, region: 1, technology: 2, year: 2)
Coordinates:
  * technology             (technology) object 'gasboiler' 'heatpump'
  * region                 (region) object 'USA'
  * year                   (year) int64 2020 2060
  * commodity              (commodity) object 'heat'
    comm_usage             (commodity) object CommodityUsage.ENERGY|PRODUCT
Data variables:
    level                  (technology, region) object 'fixed' 'fixed'
    cap_par                (technology, region) float64 3.8 8.867
    cap_exp                (technology, region) int64 1 1
    fix_par                (technology, region) int64 0 0
    fix_exp                (technology, region) int64 1 1
    var_par                (technology, region) int64 0 0
    var_exp                (technology, region) int64 1 1
    max_capacity_addition  (technology, region) int64 100 100
    max_capacity_growth    (technology, region) int64 100 100
    total_capacity_limit   (technology, region) float64 3.937e+03 3.937e+03
    technical_life         (technology, region) int64 10 10
    utilization_factor     (technology, region) int64 1 1
    scaling_size           (technology, region) float64 1.89e-06 1.89e-06
    efficiency             (technology, region) int64 86 86
    availability           (technology, region) int64 1 1
    interest_rate          (technology, region) float64 0.1 0.1
    type                   (technology, region) object 'energy' 'energy'
    fuel                   (technology) object 'gas' 'electricity'
    enduse                 (technology) object 'heat' 'heat'
    agent_share_2          (technology, region) int64 1 1
    tech_type              (technology) object 'energy' 'energy'
    fixed_outputs          (commodity, technology, region, year) float64 1.0 ... 1.0
    flexible_outputs       (commodity, technology, region, year) float64 0.0 ... 0.0
    commodity_units        (commodity) object 'PJ/PJ'
    fixed_inputs           (commodity, technology, region, year) float64 0.0 ... 0.0
    flexible_inputs        (commodity, technology, region, year) float64 0.0 ... 0.0
    comm_name              (commodity) object 'Heat'
    emmission_factor       (commodity) float64 0.0
    heat_rate              (commodity) int64 1
    unit                   (commodity) object 'PJ'

The docstrings can help determine which function does what:

In [26]:
from muse.commodities import is_pollutant

is_pollutant?